In [1]:
!pip install gtts
!pip install replicate
!pip install langchain
!pip install faiss-cpu
!pip install InstructorEmbedding
!pip install sentence_transformers
! pip install requests ffmpeg-python
! pip install deepgram-sdk --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=1926dabb5334f603c1e65590be5772b7f4b4a8005997dd

In [2]:
import os
import time
import sqlite3
import nest_asyncio
from gtts import gTTS
from io import BytesIO
import asyncio, json, os
from deepgram import Deepgram
from base64 import b64decode
from google.colab import output
from IPython.display import Audio
from IPython.display import Javascript
from langchain.vectorstores import FAISS
from langchain.llms import Replicate
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import WebBaseLoader

In [3]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

In [4]:
REPLICATE_MODEL = "replicate/llama-2-70b-chat:2c1608e18606fad2812020dc541930f2d0495ce32eee50074220b87300bc16e1"
PROMPT_TEMPLATE = """You are a helpful support chatbot having a conversation with {user_name}.
    Follow these 3 steps:
    1. Read the context below
    2. Answer the question only using the context information
    3. Show the source for your answers

    Context : {context}
    User Question: {question}

    If you don't know the answer, just say you don't know. Do NOT try to make up an answer. If the question is
    not related to the context, politely respond that you are tuned to only answer questions that are related to
    the context. Use as much detail as possible when responding.
    Also if the question is hi, politely reply as Hello! I am your helpful assistant. How may I assist you today?"""

URLS = ["https://giki.edu.pk/",
        "https://giki.edu.pk/vision-and-mission/",
        "https://giki.edu.pk/institute/",
        "https://giki.edu.pk/academics/office-of-admission-examination/",
        "https://admissions.giki.edu.pk/Home/Index",
        "https://giki.edu.pk/admissions/admissions-undergraduates/ugradhow-to-apply/",
        "https://giki.edu.pk/admissions/admissions-undergraduates/ugrad-fees-and-expenses/",
        "https://giki.edu.pk/admissions/admissions-undergraduates/",
        "https://giki.edu.pk/contact-us-main/",
        "https://giki.edu.pk/admissions/admissions-undergraduates/",
        "https://giki.edu.pk/admissions/admissions-undergraduates/ugrad-aid-scholarships/",
        "https://giki.edu.pk/admissions/admissions-graduate/",
        "https://giki.edu.pk/admissions/admissions-graduate/grad-aid-scholarships/"
]

In [5]:
DATABASE_FILE = "chatbot.db"

# Function to set up the database and tables
def setup_database():
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()

    # Create a table to store user information (username and replicate key)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_name TEXT,
            replicate_api_token TEXT
        )
    """)

    # Create a table to store user inputs and chatbot responses
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS conversations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            user_input TEXT,
            chatbot_response TEXT,
            FOREIGN KEY (user_id) REFERENCES users (user_id)
        )
    """)

    conn.commit()
    conn.close()

# Set up the database
setup_database()

In [6]:
class Chatbot:
    def __init__(self, replicate_api_token, user_name):
        self.replicate_api_token = replicate_api_token
        self.llm_chain = None
        self.memory = None
        self.user_name = user_name
        self.save_user_info()
        self.dg_key = 'f7504a4b2f1ae12136205586226f6a8bc7ce37fb'   # f7504a4b2f1ae12136205586226f6a8bc7ce37fb
        self.dg = Deepgram(self.dg_key)
        self.params = {"punctuate": True, "model": 'general', "tier": 'nova'}

    def save_user_info(self):
        conn = sqlite3.connect(DATABASE_FILE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO users (user_name, replicate_api_token) VALUES (?, ?)", (self.user_name, self.replicate_api_token))
        conn.commit()
        conn.close()

    def initialize_replicate_api(self):
        os.environ['REPLICATE_API_TOKEN'] = self.replicate_api_token

    def record_audio(self, sec=3):
        display(Javascript(RECORD))
        print('Recording...')
        s = output.eval_js('record(%d)' % (sec * 1000))
        b = b64decode(s.split(',')[1])
        with open('audio.wav', 'wb') as f:
            f.write(b)
        print('Done.')
        user_input = self.convert_audioToText()
        return user_input

    def convert_audioToText(self):
        DIRECTORY = '.'
        MIMETYPE = 'wav'
        audio_folder = os.listdir(DIRECTORY)
        for audio_file in audio_folder:
          if audio_file.endswith(MIMETYPE):
            with open(f"{DIRECTORY}/{audio_file}", "rb") as f:
              source = {"buffer": f, "mimetype":'audio/'+MIMETYPE}
              res = self.dg.transcription.sync_prerecorded(source, self.params)
              with open(f"./{audio_file[:-4]}.json", "w") as transcript:
                  json.dump(res, transcript)
        transcription_file = '/content/audio.json'
        with open(transcription_file, "r") as file:
          data = json.load(file)
          result = data['results']['channels'][0]['alternatives'][0]['transcript']
          result = result.split('.')
          question = [sentence.strip() for sentence in result]

        return question[0]

    def text_to_speech(self,text):
        tts = gTTS(text=text, lang='en')
        audio_stream = BytesIO()
        tts.write_to_fp(audio_stream)
        audio_stream.seek(0)

        return audio_stream


    def load_documents(self, urls):
        loader = WebBaseLoader(urls)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        texts = text_splitter.split_documents(docs)
        return texts

    def create_instructor_retriever(self, texts):
        instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device": "cuda"})
        db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)
        retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})
        return retriever

    def create_prompt_template(self):
        return PromptTemplate(input_variables=["question", "context","user_name"], template=PROMPT_TEMPLATE)

    def clear_memory(self):
        if self.memory is not None:
            self.memory.clear()

    def create_conversation_memory(self):
        return ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)

    def create_llm_chain(self, texts):
        llm = Replicate(model=REPLICATE_MODEL, input={"temperature": 0.1, "top_p": 0.9, "max_length": 512, "repetition_penalty": 1}, streaming=True)
        retriever = self.create_instructor_retriever(texts)
        self.memory = self.create_conversation_memory()
        self.llm_chain = ConversationalRetrievalChain.from_llm(llm, retriever, memory=self.memory)

    def initialize_chatbot(self, urls):
        self.initialize_replicate_api()
        nest_asyncio.apply()
        texts = self.load_documents(urls)
        self.create_llm_chain(texts)

    def generate_response(self, user_input):
        output = self.llm_chain({"question": user_input})
        response = ' '.join(output['answer'])
        conn = sqlite3.connect(DATABASE_FILE)
        cursor = conn.cursor()
        cursor.execute("SELECT user_id FROM users WHERE user_name=?", (self.user_name,))
        user_id = cursor.fetchone()[0]
        cursor.execute("INSERT INTO conversations (user_id, user_input, chatbot_response) VALUES (?, ?, ?)", (user_id, user_input, response))
        conn.commit()
        conn.close()
        return response

In [7]:
replicate_api_token = input('Enter Replicate API token: ').strip()
if not (replicate_api_token.startswith('r8_') and len(replicate_api_token) == 40):
  raise ValueError('Please enter a valid Replicate API token.')

user_name = input("Enter your name? ").strip()
urls = URLS

print('Initializing...')
chatbot = Chatbot(replicate_api_token, user_name)
chatbot.initialize_chatbot(urls)

print("🦙💬 Llama 2 Chatbot")
full_response = f'''Hello! Nice to meet you {user_name}! I am your helpful assistant. How may I assist you today?
Enter 'exit', 'quit', or 'bye' to end the conversation.
Enter 'clear memory' to clear the conversation memory.'''
print(full_response)
audio_stream = chatbot.text_to_speech(full_response)
display(Audio(audio_stream.read(), autoplay=True))

while True:

    user_input = chatbot.record_audio(5)
    print(f"{user_name}: {user_input}.")

    if user_input.lower() in ['exit', 'quit', 'bye']:
        response = "Goodbye! Have a nice day!"
        print(f"Assistant: {response}")
        audio_stream = chatbot.text_to_speech(response)
        display(Audio(audio_stream.read(), autoplay=True))
        break

    if user_input.lower() == 'clear':
        chatbot.clear_memory()
        response = "Conversation memory has been cleared."
        print(f"Assistant: {response}")
        audio_stream = chatbot.text_to_speech(response)
        display(Audio(audio_stream.read(), autoplay=True))
        continue

    print("Thinking...")
    response = chatbot.generate_response(user_input)
    words = response.split()
    if len(words) > 1:
        full_response = response
    else:
        full_response = response.replace(" ","")
    print(f"Assistant: {full_response}")
    audio_stream = chatbot.text_to_speech(full_response)
    display(Audio(audio_stream.read(), autoplay=True))
    time.sleep(60)

Enter Replicate API token: r8_G6U4aj41rPHPi1gnjltPm8BTZeWFzxc3MW2K2
Enter your name? cream-cheese
Initializing...


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
🦙💬 Llama 2 Chatbot
Hello! Nice to meet youcream-cheese! I am your helpful assistant. How may I assist you today?
Enter 'exit', 'quit', or 'bye' to end the conversation.
Enter 'clear memory' to clear the conversation memory.


<IPython.core.display.Javascript object>

Recording...
Done.
cream-cheese: Bye.
Assistant: Goodbye! Have a nice day!
